<a href="https://colab.research.google.com/github/sshee0123/Volleyball_Match_Prediction/blob/main/%EB%B0%B0%EA%B5%AC%EA%B2%BD%EA%B8%B0%EC%98%88%EC%B8%A1_autoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret --quiet
!pip install shap --quiet

     |████████████████████████████████| 320 kB 5.2 MB/s 
     |████████████████████████████████| 1.3 MB 43.7 MB/s 
     |████████████████████████████████| 16.8 MB 632 kB/s 
     |████████████████████████████████| 6.8 MB 39.6 MB/s 
     |████████████████████████████████| 1.7 MB 72.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0 MB 49.4 MB/s 
     |████████████████████████████████| 116 kB 55.6 MB/s 
     |████████████████████████████████| 261 kB 54.7 MB/s 
     |████████████████████████████████| 167 kB 48.9 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |████████████████████████████████| 88 kB 6.9 MB/s 
     |████████████████████████████████| 102 kB 11.2 MB/s 
     |████████████████████████████████| 303 kB 52.6 MB/s 
     |████████████████████████████████| 3.1 MB 31.6 MB/s 
     |██████████████████

# Data load

In [2]:
import pandas as pd
import numpy as np
import datetime

SEED = 42
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/volleyball_data.csv')
data.head()

,Unnamed: 0,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,...,B팀_선수4_범실_범실,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실
0,3,1,29.85,26.83,100.00,0.0,8.33,0.0,45.45,0.20,...,1.0,27.03,30.00,20.0,0.0,0.0,0.0,27.27,0.0,6.0
1,5,0,43.66,34.21,33.33,0.0,58.82,0.0,53.85,0.00,...,3.0,50.00,50.00,0.0,0.0,0.0,0.0,0.00,0.2,1.0
2,6,1,35.29,42.86,25.00,0.0,37.50,0.0,18.18,0.00,...,5.0,17.39,18.18,0.0,0.0,0.0,0.0,18.18,0.0,2.0
3,7,0,54.93,43.24,0.00,0.0,77.78,0.0,40.00,0.00,...,1.0,10.00,20.00,0.0,0.0,0.0,0.0,0.00,0.2,7.0
4,8,1,39.02,29.41,0.00,0.0,56.25,0.0,33.33,0.33,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,1.0


In [22]:
# # 결과 열을 제외한 열은 X, 결과 열은 y에 저장
# X = data.drop('결과', axis = 1)
# y = data['결과']

In [23]:
# # 학습, 검증 데이터 분리
# from sklearn.model_selection import train_test_split

# SEED = 42
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state=SEED)

In [8]:
from pycaret.classification import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# 셋업

In [ ]:
# clf = setup(data = train, target = '결과',
#             session_id = SEED,
#             numeric_features=numeric_features,           
#             categorical_features=categorical_features,
#             ignore_features=ignore_features,           
#             categorical_imputation='mode',
#             train_size=0.8,
#             profile = True # 분석을 끝내고 나서 보고서 생성
#             )

In [10]:
setup_clf = setup(data=data, target='결과', profile = True)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
setup_clf

(False,
      Unnamed 0  A팀_선수1_공격종합_성공률  A팀_선수1_오픈_성공률  A팀_선수1_시간차_성공률  A팀_선수1_이동_성공률  \
 0          3.0        29.850000      26.830000      100.000000            0.0   
 1          5.0        43.660000      34.209999       33.330002            0.0   
 2          6.0        35.290001      42.860001       25.000000            0.0   
 3          7.0        54.930000      43.240002        0.000000            0.0   
 4          8.0        39.020000      29.410000        0.000000            0.0   
 ..         ...              ...            ...             ...            ...   
 342      382.0        42.110001      33.330002      100.000000            0.0   
 343      383.0        48.279999      50.000000       75.000000            0.0   
 344      384.0        35.380001      37.840000        0.000000            0.0   
 345      385.0        26.670000       0.000000        0.000000            0.0   
 346      386.0        65.629997      75.000000        0.000000            0.0   
 
      

In [12]:
top3 = compare_models(sort='Accuracy', n_select=3,fold=5)
top3

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6116,0.6327,0.6000,0.6117,0.6041,0.2223,0.2238,0.216
lightgbm,Light Gradient Boosting Machine,0.6077,0.6409,0.5750,0.6092,0.5873,0.2136,0.2166,0.124
ridge,Ridge Classifier,0.5948,0.0000,0.5833,0.6022,0.5891,0.1898,0.1922,0.018
lda,Linear Discriminant Analysis,0.5866,0.5869,0.5833,0.5876,0.5839,0.1734,0.1742,0.026
et,Extra Trees Classifier,0.5708,0.5797,0.5417,0.5643,0.5513,0.1403,0.1402,0.544
svm,SVM - Linear Kernel,0.5576,0.0000,0.4667,0.5601,0.4843,0.1144,0.1135,0.020
lr,Logistic Regression,0.5535,0.5579,0.5417,0.5581,0.5473,0.1071,0.1078,1.280
rf,Random Forest Classifier,0.5501,0.5734,0.5167,0.5437,0.5284,0.0990,0.0992,0.584
ada,Ada Boost Classifier,0.5369,0.5623,0.4750,0.5390,0.5018,0.0719,0.0727,0.130
knn,K Neighbors Classifier,0.5207,0.5323,0.5083,0.5140,0.5098,0.0413,0.0415,0.140


[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=5777, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=3

In [13]:
tuned_top3 = [tune_model(i) for i in top3]
tuned_top3 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6800,0.0,0.5000,0.7500,0.6000,0.3506,0.3707
1,0.4800,0.0,0.4167,0.4545,0.4348,-0.0450,-0.0452
2,0.4583,0.0,0.2500,0.4286,0.3158,-0.0833,-0.0917
3,0.7917,0.0,0.8333,0.7692,0.8000,0.5833,0.5854
4,0.5833,0.0,0.8333,0.5556,0.6667,0.1667,0.1925
5,0.5417,0.0,0.5000,0.5455,0.5217,0.0833,0.0836
6,0.5417,0.0,0.5833,0.5385,0.5600,0.0833,0.0836
7,0.5417,0.0,0.6667,0.5333,0.5926,0.0833,0.0861
8,0.7083,0.0,0.6667,0.7273,0.6957,0.4167,0.4181


[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.5, loss='deviance', max_depth=3,
                            max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.0002,
                            min_impurity_split=None, min_samples_leaf=5,
                            min_samples_split=7, min_weight_fraction_leaf=0.0,
                            n_estimators=240, n_iter_no_change=None,
                            presort='deprecated', random_state=5777,
                            subsample=0.7, tol=0.0001, validation_fraction=0.1,
                            verbose=0, warm_start=False),
 LGBMClassifier(bagging_fraction=0.4, bagging_freq=1, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=16, min_child_weight=0.0

In [15]:
blender_top3 = blend_models(estimator_list=tuned_top3, verbose=True, method='auto')
blender_top3

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6400,0.0,0.5833,0.6364,0.6087,0.2765,0.2774
1,0.4800,0.0,0.4167,0.4545,0.4348,-0.0450,-0.0452
2,0.5417,0.0,0.2500,0.6000,0.3529,0.0833,0.1026
3,0.8333,0.0,0.7500,0.9000,0.8182,0.6667,0.6761
4,0.6667,0.0,1.0000,0.6000,0.7500,0.3333,0.4472
5,0.5417,0.0,0.5000,0.5455,0.5217,0.0833,0.0836
6,0.5833,0.0,0.6667,0.5714,0.6154,0.1667,0.1690
7,0.5833,0.0,0.6667,0.5714,0.6154,0.1667,0.1690
8,0.7917,0.0,0.9167,0.7333,0.8148,0.5833,0.6025


VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.5,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features='sqrt',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0002,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=5,
                                                         min_samples_split=7,
                                        

In [25]:
blender_tune3 = tune_model(blender_top3)
blender_tune3

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7600,0.0,0.6667,0.8000,0.7273,0.5161,0.5230
1,0.4400,0.0,0.3333,0.4000,0.3636,-0.1290,-0.1307
2,0.5417,0.0,0.2500,0.6000,0.3529,0.0833,0.1026
3,0.6667,0.0,0.5833,0.7000,0.6364,0.3333,0.3381
4,0.6250,0.0,0.7500,0.6000,0.6667,0.2500,0.2582
5,0.6250,0.0,0.6667,0.6154,0.6400,0.2500,0.2509
6,0.6667,0.0,0.6667,0.6667,0.6667,0.3333,0.3333
7,0.6250,0.0,0.6667,0.6154,0.6400,0.2500,0.2509
8,0.7500,0.0,1.0000,0.6667,0.8000,0.5000,0.5774


VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.5,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features='sqrt',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0002,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=5,
                                                         min_samples_split=7,
                                        

# 모델 예측하기

In [17]:
# final_model = finalize_model(blender_tune3)
# prediction = predict_model(final_model, data=data)
# prediction

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8444,0.844,0.8333,0.8434,0.8383,0.6883,0.6884


,Unnamed: 0,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,...,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실,Label
0,3,1,29.85,26.83,100.00,0.0,8.33,0.0,45.45,0.20,...,27.03,30.00,20.0,0.0,0.0,0.00,27.27,0.00,6.0,1
1,5,0,43.66,34.21,33.33,0.0,58.82,0.0,53.85,0.00,...,50.00,50.00,0.0,0.0,0.0,0.00,0.00,0.20,1.0,0
2,6,1,35.29,42.86,25.00,0.0,37.50,0.0,18.18,0.00,...,17.39,18.18,0.0,0.0,0.0,0.00,18.18,0.00,2.0,0
3,7,0,54.93,43.24,0.00,0.0,77.78,0.0,40.00,0.00,...,10.00,20.00,0.0,0.0,0.0,0.00,0.00,0.20,7.0,0
4,8,1,39.02,29.41,0.00,0.0,56.25,0.0,33.33,0.33,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,382,1,42.11,33.33,100.00,0.0,38.46,0.0,66.67,0.33,...,30.77,0.00,100.0,0.0,0.0,0.00,75.00,0.25,2.0,1
343,383,1,48.28,50.00,75.00,0.0,60.00,0.0,25.00,0.00,...,50.00,100.00,0.0,0.0,0.0,50.00,0.00,0.00,2.0,0
344,384,0,35.38,37.84,0.00,0.0,26.09,0.0,60.00,0.25,...,55.56,60.00,100.0,0.0,0.0,0.00,33.33,0.25,2.0,0
345,385,0,26.67,0.00,0.00,0.0,0.00,0.0,50.00,0.00,...,20.00,20.00,0.0,0.0,0.0,0.00,0.00,0.25,2.0,0


In [27]:
final_model = finalize_model(blender_tune3)
prediction = predict_model(final_model, data=data.tail(100))
prediction

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.85,0.8477,0.8085,0.8636,0.8352,0.6978,0.6991


,Unnamed: 0,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,...,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실,Label
247,281,1,50.00,45.45,0.0,0.0,57.14,0.0,58.33,0.00,...,44.44,50.00,0.00,25.00,0.0,66.67,0.00,0.00,1.0,1
248,282,0,40.79,39.39,0.0,0.0,44.44,0.0,33.33,0.00,...,41.18,40.00,66.67,33.33,0.0,33.33,0.00,0.00,0.0,0
249,283,0,37.84,42.11,0.0,0.0,33.33,0.0,33.33,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.25,1.0,0
250,284,0,34.78,26.19,80.0,0.0,39.02,0.0,25.00,0.00,...,25.00,25.00,0.00,0.00,0.0,0.00,0.00,0.20,2.0,0
251,285,0,24.29,23.68,50.0,0.0,28.57,0.0,11.11,0.00,...,27.59,21.05,0.00,0.00,0.0,0.00,40.00,0.20,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,382,1,42.11,33.33,100.0,0.0,38.46,0.0,66.67,0.33,...,30.77,0.00,100.00,0.00,0.0,0.00,75.00,0.25,2.0,1
343,383,1,48.28,50.00,75.0,0.0,60.00,0.0,25.00,0.00,...,50.00,100.00,0.00,0.00,0.0,50.00,0.00,0.00,2.0,0
344,384,0,35.38,37.84,0.0,0.0,26.09,0.0,60.00,0.25,...,55.56,60.00,100.00,0.00,0.0,0.00,33.33,0.25,2.0,0
345,385,0,26.67,0.00,0.0,0.0,0.00,0.0,50.00,0.00,...,20.00,20.00,0.00,0.00,0.0,0.00,0.00,0.25,2.0,0


In [28]:
prediction = prediction['Label']
prediction

247    1
248    0
249    0
250    0
251    1
      ..
342    1
343    0
344    0
345    0
346    0
Name: Label, Length: 100, dtype: int64

# 평가

In [30]:
from pycaret.utils import check_metric
check_metric(data['결과'].tail(100), prediction, metric = 'Accuracy')

0.85